In [ ]:
folderPath = "D:/"

# Load data helpers

In [ ]:
import numpy as np
import pandas as pd

def PrintGreen(text):
    print('\x1b[6;30;42m' + text + '\x1b[0m')
    
def PrintRed(text):
    print('\33[41m' + text + '\x1b[0m')

def LoadData(filename, rowsName, columnsName):
    newDataframe = pd.read_csv(filename, na_values = 'null')
    if newDataframe.shape[0] > 0 and newDataframe.shape[1] > 0:
        PrintGreen("Loading " + filename + " succeeded");
    else:
        PrintRed("Loading " + filename + " failed!");

    print(rowsName + " = " + str(newDataframe.shape[0]))
    print(columnsName + " = " + str(newDataframe.shape[1]))

    return newDataframe

# Ensure to show all columns
pd.set_option('display.max_columns', None)

## Motion Database Poses
Evaluated skeletal poses at a given sample rate. Position and rotation in local space. Rotation is represented as the X and Y basis vectors of the transform.

In [ ]:
dataPosRotModelSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_PosRot_ModelSpace_60Hz.csv', "Frames", "PoseComponents")
dataPosRotModelSpace.head(15)

In [ ]:
dataRotModelSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_Rot_ModelSpace_60Hz.csv', "Frames", "PoseComponents")
dataRotModelSpace.head(15)

In [ ]:
dataPosRotLocalSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_PosRot_LocalSpace_60Hz.csv', "Frames", "PoseComponents")
dataPosRotLocalSpace.head(15)

In [ ]:
dataRotLocalSpace = LoadData(folderPath + 'MotionMatchingDatabase_Poses_Rot_LocalSpace_60Hz.csv', "Frames", "PoseComponents")
dataRotLocalSpace.head(15)

## Motion Database Features
Feature matrix

In [ ]:
dataFeatures = LoadData(folderPath + 'MotionMatchingDatabase_Features_60Hz.csv', "Frames", "FeatureComponents")

if (dataPosRotLocalSpace.shape[0] == dataFeatures.shape[0]
    and dataPosRotModelSpace.shape[0] == dataFeatures.shape[0]
    and dataRotLocalSpace.shape[0] == dataFeatures.shape[0]
    and dataRotModelSpace.shape[0] == dataFeatures.shape[0]):
    PrintGreen("Frame numbers match.")
else:
    PrintRed("Frame numbers do not match!")

dataFeatures.head(50)

## Recorded data (Poses, features and best matching frames)

In [ ]:
recording_poses = LoadData(folderPath + 'ReallyLongRuntimeRecordings/RuntimeRecording_Poses.csv', "Frames", "PoseComponents")
recording_rotations = LoadData(folderPath + 'ReallyLongRuntimeRecordings/RuntimeRecording_Rotations.csv', "Frames", "RotationComponents")
recording_features = LoadData(folderPath + 'ReallyLongRuntimeRecordings/RuntimeRecording_Features.csv', "Frames", "FeatureComponents")
recording_bestMatchFrames = LoadData(folderPath + 'ReallyLongRuntimeRecordings/RuntimeRecording_BestMatchingFrames.csv', "Frames", "BestMatchingFrameComponents")

if (recording_poses.shape[0] == recording_rotations.shape[0] == recording_features.shape[0] == recording_bestMatchFrames.shape[0]):
    PrintGreen("Frame numbers match.")
else:
    PrintRed("Frame numbers do not match!")

In [ ]:
recordingPoseData = np.delete(recording_poses.values, -1, axis=0) # remove last frame
recordingFeatureData = np.delete(recording_features.values, -1, axis=0) # remove last frame
print(recordingPoseData.shape)
print(recordingFeatureData.shape)
recording_input = np.concatenate((recordingPoseData, recordingFeatureData), axis=1) # concatenate horizontally
print(recording_input.shape)

recordingOutputPoseData = np.delete(recording_rotations.values, 0, axis=0)
print(recordingOutputPoseData.shape)


## Data Preparation / Normalization

In [ ]:
# from sklearn import preprocessing
# from sklearn.preprocessing import StandardScaler

# posRotModelSpaceDataScaler = preprocessing.StandardScaler()
# dataPosRotModelSpaceScaled = posRotModelSpaceDataScaler.fit_transform(dataPosRotModelSpace.values)

# rotModelSpaceDataScaler = preprocessing.StandardScaler()
# dataRotModelSpaceScaled = rotModelSpaceDataScaler.fit_transform(dataRotModelSpace.values)

# posRotLocalSpaceDataScaler = preprocessing.StandardScaler()
# dataPosRotLocalSpaceScaled = posRotLocalSpaceDataScaler.fit_transform(dataPosRotLocalSpace.values)

# rotLocalSpaceDataScaler = preprocessing.StandardScaler()
# dataRotLocalSpaceScaled = rotLocalSpaceDataScaler.fit_transform(dataRotLocalSpace.values)

# featureDataScaler = preprocessing.StandardScaler()
# dataFeaturesScaled = featureDataScaler.fit_transform(dataFeatures.values)

# # Input (Pose + Features)
# inputPoseData = np.delete(dataPosRotLocalSpaceScaled, -1, axis=0) # remove last frame
# inputFeatureData = np.delete(dataFeaturesScaled, -1, axis=0) # remove last frame
# print(inputPoseData.shape)
# print(inputFeatureData.shape)
# data_input = np.concatenate((inputPoseData, inputFeatureData), axis=1) # concatenate horizontally
# print(data_input.shape)

# # Output (Next pose)
# data_output = dataRotLocalSpaceScaled
# data_output = np.delete(data_output, 0, axis=0) # remove first frame
# print(data_output.shape)

# Input (Pose + Features)
# inputPoseData = np.delete(dataPosRotLocalSpace.values, -1, axis=0) # remove last frame
# inputFeatureData = np.delete(dataFeatures.values, -1, axis=0) # remove last frame
# print(inputPoseData.shape)
# print(inputFeatureData.shape)
# data_input = np.concatenate((inputPoseData, inputFeatureData), axis=1) # concatenate horizontally
# print(data_input.shape)

# # Output (Next pose)
# outputPoseData = np.delete(dataRotLocalSpace.values, 0, axis=0)
# # outputFeatureData = np.delete(dataFeatures.values, 0, axis=0)
# # print(outputPoseData.shape)
# # print(outputFeatureData.shape)
# # data_output = np.concatenate((outputPoseData, outputFeatureData), axis=1) # concatenate horizontally
# data_output = outputPoseData
# print(data_output.shape)

# data_input = np.concatenate((data_input, recording_input), axis=0)
# print(data_input.shape)

# data_output = np.concatenate((data_output, recordingOutputPoseData), axis=0)
# print(data_output.shape)

data_input = recording_input
print(data_input.shape)

data_output = recordingOutputPoseData
print(data_output.shape)

In [ ]:
import matplotlib.pyplot as plt

def GetBadFrames(frameData):
    sum_square_diff = np.empty(len(frameData)-1)
    badFrames = []
    for i in range(1, len(frameData)):
        sum_square_diff[i-1] = np.linalg.norm((frameData[i] - frameData[i-1]))
        if sum_square_diff[i-1] > 4:
            badFrames.append(i-1)
    plt.figure(figsize=(20,10))
    plt.plot(sum_square_diff)
    plt.show
    return badFrames

inputFrameData = data_input
badFrames = GetBadFrames(inputFrameData)
print(len(badFrames))

data_input = np.delete(data_input, badFrames, axis=0)
data_output = np.delete(data_output, badFrames, axis=0)
print(len(data_input))
print(len(data_output))

In [ ]:
len(data_input)

In [ ]:
# # Convert Pandas DataFrames to PyTorch tensors
# X = torch.tensor(data_input, dtype=torch.float32)
# y = torch.tensor(data_output, dtype=torch.float32)

## ML

In [ ]:
# Tensorboard
import torch
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

logdir = folderPath + "TensorBoard/" # + datetime.now().strftime("%Y%m%d") + "/"
tensorBoardWriter = SummaryWriter(logdir)

## PyTorch Model

In [ ]:
import torch
import torch.nn as nn

# Set fixed random number seed
torch.manual_seed(42)

def ConstructModel(inputFeatures, outputFeatures):
    hidden_layer = 512
    model = nn.Sequential(nn.Linear(inputFeatures, hidden_layer),
                          nn.ReLU(),
                          #nn.Dropout(0.1), # probability of an element to be zeroed. Default: 0.5
                          #nn.BatchNorm1d(layer2),
                          nn.Linear(hidden_layer, hidden_layer),
                          nn.ReLU(), #nn.LeakyReLU(),
                          #nn.Dropout(0.1),
                          #nn.BatchNorm1d(layer1),
                          nn.Linear(hidden_layer, outputFeatures),
                          #nn.Sigmoid()
                          #nn.ReLU()
                          #nn.Tanh()
                          )
    
    def init_weights(model):
        if isinstance(model, nn.Linear):
            torch.nn.init.xavier_uniform(model.weight)
            model.bias.data.fill_(0.01)

    init_weights(model)
    print(model)
    return model

## Export PyTorch model to an ONNX file

In [ ]:
from torch import nn
import torch.utils.model_zoo as model_zoo
import torch.onnx

def ExportModelToOnnx(model, filename, dummy_input):
    torch.onnx.export(model,                     # model being run
                      dummy_input,               # model input (or a tuple for multiple inputs)
                      filename,                  # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=10,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

## Training

In [ ]:
# Settings
learning_rate = 1e-3
weight_decay = 1e-5
epochs = 3
loss_function = nn.MSELoss()
mini_batch_size = 32
# Note: Optimizer can be changed in the code below (as it needs the model parameters it can't be done here)

In [ ]:
# def loss_function(input, target):
#     total = 0
#     for batch_index in range(len(input)):
#         diff = input[batch_index] - target[batch_index]
#         for i in range(1098, 1157):
#             diff[i] *= 10
#         total += (diff**2).mean()
#     return total

# def loss_function(input, target):
#     total = 0
#     for batch_index in range(len(input)):
#         diff = (input[batch_index] - target[batch_index])**2
#         for i in range(1098, 1157):
#             diff[i] *= 20
#         total += diff.mean()
#     return total

In [ ]:
from random import shuffle
from torch.utils.data import Dataset
import matplotlib.pyplot as plt

# Convert an input and output tensor into a dataset
# Inputs:
# X: torch.tensor specifying the training input data
# y: torch.tensor specifying the training labels
class CreatePytorchDataset(Dataset):
    def __getitem__(self,idx):
        return self.x_train[idx],self.y_train[idx]
    def __init__(self,X, y):
        self.x_train=X
        self.y_train=y 
    def __len__(self):
        return len(self.y_train)

def Training(model, train_loader, test_loader, optimizer):

    timeCode = datetime.now().strftime("%H:%M:%S") # used for TensorBoard

    # Re-train on the same data for the given amount of epochs
    losses = []
    for epoch in range(0, epochs):

        print("Epoch " + str(epoch) + ": ")
        running_loss = 0.0
        current_mini_batch = 0

        # Mini-batches
        for i, data in enumerate(train_loader, 0):

            # Get the inputs and labels/outputs from the dataset
            X, y = data

            #################
            # Backpropagation

            # Zero the gradients
            model.zero_grad()

            # Perform forward pass and compute prediction
            pred_y = model(X)

            # Compute loss
            loss = loss_function(pred_y, y)
            running_loss += loss.item()
            tensorBoardWriter.add_scalar(timeCode + " Loss / train", loss.item(), current_mini_batch)

            # Perform backward pass
            loss.backward()

            # Perform optimization
            optimizer.step()

            # Print statistics
            print_each = 100
            if current_mini_batch % print_each == print_each - 1:
                averaged_loss = running_loss / print_each
                print("Loss after mini-batch %5d: %.5f" % (current_mini_batch + 1, averaged_loss))
                tensorBoardWriter.add_scalar(timeCode + " Running loss / train", averaged_loss, current_mini_batch)
                losses.append(averaged_loss)
                running_loss = 0.0

            current_mini_batch = current_mini_batch + 1


    lossData = pd.DataFrame(losses)
    lossData.to_csv("D:/OnnxModelLosses_PosRotFeaturesWithHandAndHeadLocal_To_RotLocal_Unnormalized_NoBadFrames_Shuffled_LR1e-3_Batch32_Hidden512_WeightDecay1e-5_Epochs3_NormalizeDataOff_260kRecordedONLY_Threshold4.csv", index=False)
    plt.plot(losses)
    plt.ylabel('loss')
    plt.xlabel('mini-batch')
    plt.title("Learning rate %f"%(learning_rate))
    plt.show()

    # TensorBoard
    tensorBoardWriter.close()


# Convert Pandas DataFrames to PyTorch tensors
X = torch.tensor(data_input, dtype=torch.float32)
y = torch.tensor(data_output, dtype=torch.float32)

# Construct the PyTorch dataset and data loaders
torchDataset = CreatePytorchDataset(X, y)
train_loader = torch.utils.data.DataLoader(torchDataset, batch_size=mini_batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torchDataset, batch_size=mini_batch_size, shuffle=True)

# Construct the model and optimizer and train the model
inputFeatures = X.shape[1]
outputFeatures = y.shape[1]
model = ConstructModel(inputFeatures, outputFeatures)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
Training(model, train_loader, test_loader, optimizer)

# Export the trained model to an ONNX file
filename = folderPath + 'OnnxModel_PosRotFeaturesWithHandAndHeadLocal_To_RotLocal_Unnormalized_NoBadFrames_Shuffled_LR1e-3_Batch32_Hidden512_WeightDecay1e-5_Epochs3_NormalizeDataOff_260kRecordedONLY_Threshold4.onnx'
ExportModelToOnnx(model, filename, X)

In [ ]:
import onnxruntime

ort_session = onnxruntime.InferenceSession("D:/OnnxModel_PosRotFeaturesWithHandAndHeadLocal_To_RotLocal_Unnormalized_NoBadFrames_Shuffled_LR1e-3_Batch32_Hidden512_WeightDecay1e-5_Epochs3_NormalizeDataOff_260kRecordedONLY_Threshold4.onnx")

# compute ONNX Runtime output prediction
# ort_input = {"input": [X[0].numpy()]}
X = torch.tensor(np.concatenate((dataPosRotLocalSpace.values, dataFeatures.values), axis=1), dtype=torch.float32)
ort_input = {"input": X.numpy()}
ort_outs = ort_session.run(None, ort_input)

In [ ]:
ort_outs[0][25][25]

In [ ]:
torch_outs = model(X)

In [ ]:
len(torch_outs)

In [ ]:
# referencePoses = np.concatenate((dataRotLocalSpace.values, dataFeatures.values), axis=1)
referencePoses = dataRotLocalSpace.values
# referencePoses = dataRotLocalSpace.values
referencePoses

In [ ]:
l1 = np.empty(len(ort_outs[0])-1)
for i in range(len(ort_outs[0])-1):
    l1[i] = np.linalg.norm((ort_outs[0][i] - referencePoses[i + 1]), ord=1)
print("Avg L1: " + str(np.average(l1)))
plt.figure(figsize=(20,10))
plt.plot(l1)
plt.show

In [ ]:
l2 = np.empty(len(ort_outs[0])-1)
for i in range(len(ort_outs[0])-1):
    l2[i] = np.linalg.norm((ort_outs[0][i] - referencePoses[i + 1]))
print("Avg L2: " + str(np.average(l2)))
plt.figure(figsize=(20,10))
plt.plot(l2)
plt.show

In [ ]:
finalPoses = pd.DataFrame(ort_outs[0], columns=dataRotLocalSpace.columns.values)
finalPoses

In [ ]:
finalPoses.to_csv("D:/InferencedPoses_PosRotFeaturesWithHandAndHeadLocal_To_RotLocal_Unnormalized_NoBadFrames_Shuffled_LR1e-3_Batch32_Hidden512_WeightDecay1e-5_Epochs3_NormalizeDataOff_260kRecordedONLY_Threshold4.csv", index=False)

Test 1:

result_data

iterate through data_input
    resulting_scaled_pose = model.predict(data_input[i])
    resulting_pose = poseDataScaler.inverse_transform(resulting_scaled_pose)
    result_data.add_row(resulting_pose)

store_to_csv(result_data)




Result: csv file hopefully containing a similar result as the original animation

* New AnimGraphNode that can read a csv file and play it back (by applying the next pose with each Update())















TODO:
* Infer the model on all input frames, store the results inside a new .CSV file
* Create a way to play back the "animation" of the exported CSV file
* (If that looks a bit OK), use ONNX in motion matching code to step forward
* motion borders are an issue. inside the motion database we have several individual motions, but we don't separate them yet here